In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
from pathlib import Path
import zipfile
import requests

data_path = Path("data/")
text_path = data_path / "bbc"
if text_path.is_dir():
    print(f"{text_path} directory exists.")
else:
    print(f"Did not find {text_path} directory, creating one...")
    text_path.mkdir(parents=True, exist_ok=True)

    with open(data_path / "bbc.zip", "wb") as f:
        request = requests.get("https://github.com/HoussemBeltifa/Topic_Prediction/raw/main/bbc.zip")
        print("Downloading data...")
        f.write(request.content)

    with zipfile.ZipFile(data_path / "bbc.zip", "r") as zip_ref:
        print("Unzipping data...")
        zip_ref.extractall(text_path)


data/bbc directory exists.


In [ ]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of articles (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} article in '{dirpath}'.")

In [ ]:
walk_through_dir(text_path)

There are 1 directories and 0 article in 'data/bbc'.
There are 2 directories and 0 article in 'data/bbc/bbc'.
There are 5 directories and 0 article in 'data/bbc/bbc/train'.
There are 0 directories and 333 article in 'data/bbc/bbc/train/politics'.
There are 0 directories and 320 article in 'data/bbc/bbc/train/tech'.
There are 0 directories and 408 article in 'data/bbc/bbc/train/business'.
There are 0 directories and 408 article in 'data/bbc/bbc/train/sport'.
There are 0 directories and 308 article in 'data/bbc/bbc/train/entertainment'.
There are 5 directories and 0 article in 'data/bbc/bbc/test'.
There are 0 directories and 84 article in 'data/bbc/bbc/test/politics'.
There are 0 directories and 81 article in 'data/bbc/bbc/test/tech'.
There are 0 directories and 102 article in 'data/bbc/bbc/test/business'.
There are 0 directories and 103 article in 'data/bbc/bbc/test/sport'.
There are 0 directories and 78 article in 'data/bbc/bbc/test/entertainment'.


In [ ]:
# Setup train and testing paths
train_dir = text_path / "bbc" / "train"
test_dir = text_path / "bbc" / "test"

train_dir, test_dir

(PosixPath('data/bbc/bbc/train'), PosixPath('data/bbc/bbc/test'))

In [ ]:
!pip install nltk
!pip install scikit-learn

In [ ]:
import os

def load_data_from_folder(folder_path):
    categories = os.listdir(folder_path)
    data = []
    for category in categories:
        category_path = os.path.join(folder_path, category)
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data.append((file.read(), category))
    return data

train_data = load_data_from_folder(train_dir)
test_data = load_data_from_folder(test_dir)


In [ ]:
train_data[800][0]

'Weak dollar hits Reuters\n\nRevenues at media group Reuters slipped 11% during 2004, mainly due to the weakness of the dollar, the group said.\n\nThe company said it was optimistic about growth even as revenues slipped 11% from £3.24bn ($6.13bn) in 2003 to £2.89bn in 2004. Reuters beat profit forecasts, posting a 52% rise in profits for the year to £198m from the £130m seen a year earlier. Reuters also beat its savings target for 2004, delivering £234m of cuts. Under its three-year Fast Forward turnaround plan it had aimed to save £220m during the 12 months to 31 December.\n\nReuters also managed to slow a decline in underlying revenues to 5.4% from 10.2% in 2003 and cut its debt back to £160m from £610m a year earlier. The news and financial data seller said the year had begun well, adding it expected "further gradual improvement" in the second quarter of the year after good January sales. It added it was planning to deliver a further £105m of savings over the coming year - but said 

In [ ]:
# Lowercase the text data while preserving the structure
train_data_lowercased = [(text.lower(), label) for text, label in train_data]
test_data_lowercased = [(text.lower(), label) for text, label in test_data]

In [ ]:
train_data_lowercased[800][0]

'weak dollar hits reuters\n\nrevenues at media group reuters slipped 11% during 2004, mainly due to the weakness of the dollar, the group said.\n\nthe company said it was optimistic about growth even as revenues slipped 11% from £3.24bn ($6.13bn) in 2003 to £2.89bn in 2004. reuters beat profit forecasts, posting a 52% rise in profits for the year to £198m from the £130m seen a year earlier. reuters also beat its savings target for 2004, delivering £234m of cuts. under its three-year fast forward turnaround plan it had aimed to save £220m during the 12 months to 31 december.\n\nreuters also managed to slow a decline in underlying revenues to 5.4% from 10.2% in 2003 and cut its debt back to £160m from £610m a year earlier. the news and financial data seller said the year had begun well, adding it expected "further gradual improvement" in the second quarter of the year after good january sales. it added it was planning to deliver a further £105m of savings over the coming year - but said 

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# Tokenize the lowercased data
train_tokens = [(word_tokenize(text), label) for text, label in train_data_lowercased]
test_tokens = [(word_tokenize(text), label) for text, label in test_data_lowercased]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train_tokens[800][0]

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords

# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from tokenized data
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

# Remove stopwords from tokenized data
train_tokens_no_stopwords = [(remove_stopwords(tokens), label) for tokens, label in train_tokens]
test_tokens_no_stopwords = [(remove_stopwords(tokens), label) for tokens, label in test_tokens]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_tokens_no_stopwords[800][0]

In [ ]:
import re

# Function to remove special characters from a list of strings
def remove_special_characters(texts):
    cleaned_texts = []
    # Define the pattern to match special characters
    pattern = r'[^a-zA-Z\s]'
    for text in texts:
        cleaned_text = re.sub(pattern, '', text).strip()
        if cleaned_text:  # Check if the cleaned text is not empty
            cleaned_texts.append(cleaned_text)
    return cleaned_texts

train_data_f = [(remove_special_characters(text), label) for text, label in train_tokens_no_stopwords]
test_data_f = [(remove_special_characters(text), label) for text, label in test_tokens_no_stopwords]

In [ ]:
train_data_f[800][1]

'business'

In [ ]:
# Get class names as a list
a = [label for text,label in train_data_f]
class_names = list(dict.fromkeys(sorted(a)))
class_names

['business', 'entertainment', 'politics', 'sport', 'tech']

In [ ]:
# Can also get class names as a dict
class_names_dict = {label: index for index, label in enumerate(sorted(set(label for _, label in train_data_f)))}

class_names_dict

{'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}

In [ ]:
# Check the lengths
len(train_data_f), len(test_data_f)

(1777, 448)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


# Create vocabulary and label mapping
all_tokens = [token for tokens, _ in train_data_f + test_data_f for token in tokens]
vocab = {token: index for index, token in enumerate(set(all_tokens))}
label_vocab = {label: index for index, label in enumerate(sorted(set(label for _, label in train_data_f)))}

# Convert tokenized text to numericalized tensors
numericalized_texts = [[vocab[token] for token in tokens] for tokens, _ in train_data_f]
numericalized_labels = [label_vocab[label] for _, label in train_data_f]

# Define the model
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(Model, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, mode='mean')
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc5 = nn.Linear(hidden_dim, output_dim)
        self.fc3 = nn.ReLU()
        self.fc4 = nn.Dropout(p=0.3)

    def forward(self, text):
        embedded = self.embedding(text)
        x=self.fc1(embedded)
        x=self.fc3(x)
        x=self.fc4(x)
        x=self.fc2(x)
        x=self.fc3(x)
        x=self.fc4(x)
        x=self.fc5(x)
        return x

# Initialize the model
model = Model(vocab_size=len(vocab), embedding_dim=100, hidden_dim=128, output_dim=len(label_vocab))


In [ ]:
len(vocab)

30239

In [ ]:
len(label_vocab)

5

In [ ]:
from tqdm.auto import tqdm

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

epochs = 10
# Training loop
for epoch in tqdm(range(epochs)):
    running_loss = 0.0
    for text, label in zip(numericalized_texts, numericalized_labels):
        optimizer.zero_grad()
        text = torch.tensor(text, dtype=torch.long)
        output = model(text.unsqueeze(0))
        loss = criterion(output, torch.tensor([label], dtype=torch.long))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_data)}")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1, Loss: 0.5903455453967003
Epoch 2, Loss: 0.41815353043278813
Epoch 3, Loss: 0.20901931908118263
Epoch 4, Loss: 0.0880535503229168
Epoch 5, Loss: 0.0424029736468469
Epoch 6, Loss: 0.02051249118820532
Epoch 7, Loss: 0.024568708957248133
Epoch 8, Loss: 0.015426318408975884
Epoch 9, Loss: 0.0037331727075466214
Epoch 10, Loss: 0.00012205823438959998


In [ ]:
# Test loop
model.eval()

correct_predictions = 0
total_samples = len(test_data_f)

with torch.no_grad():
    for text, label in test_data_f:
        text = torch.tensor([vocab[token] for token in text], dtype=torch.long)
        output = model(text.unsqueeze(0))
        predicted_label = torch.argmax(output).item()
        true_label = label_vocab[label]

        if predicted_label == true_label:
            correct_predictions += 1

accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 96.21%


In [ ]:
import torch
import torch.nn.functional as F

def predict_topic(model, article, vocab, label_vocab):
    # Tokenize the article
    text = article.lower()
    text = word_tokenize(text)
    text = remove_stopwords(text)
    text = remove_special_characters(text)

    # Filter out unknown words
    text = [token for token in text if token in vocab]

    # Convert tokens to numericalized tensor
    numericalized_tokens = torch.tensor(
        [vocab[token] for token in text], dtype=torch.long)

    # Check if there are any tokens left after filtering out unknown words
    if len(numericalized_tokens) == 0:
        raise ValueError("All words in the article are unknown to the model.")

    # Pass the tensor through the model
    model.eval()
    with torch.no_grad():
        output = model(numericalized_tokens.unsqueeze(0))

    # Interpret the model's output
    predicted_class_index = torch.argmax(output).item()
    predicted_topic = list(label_vocab.keys())[list(
        label_vocab.values()).index(predicted_class_index)]

    return predicted_topic


In [ ]:

article = "DaimlerChrysler's 2004 sales rise\nUS-German carmaker DaimlerChrysler has sold 2.1% more cars in 2004 than in the previous year, as solid Chrysler sales offset a weak showing for Mercedes.\nSales totalled 3.9 million units worldwide during 2004, the company said at the Detroit Motor Show. A switch to new models hit luxury marque Mercedes-Benz, with sales down 3.1% at 1.06 million. Chrysler avoided the fate of US rivals Ford and General Motors, both of whom lost ground to Japanese firms. Its sales rose 3.5% to 2.7 million units.\nSimilarly on the up was the Smart brand of compact cars, with the division's sales jumping by 21.1% during 2004 to 136,000. The future of the brand - which is controlled by the Mercedes group within DaimlerChrysler - remains in question, however. Smart has consistently lost money since it started trading in 1998, and new model launches are now 'on hold', said Mercedes chief executive Eckhard Cordes. In Europe, the Smart will now go on sale through regular Mercedes dealerships as well as its own dealer network, Mr Cordes said."

predicted_topic = predict_topic(model, vocab, label_vocab, article)
print(f"Predicted Topic: {predicted_topic}")


Predicted Topic: business


In [ ]:
#save the model
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
import pickle

# save vocab and label_vocab
with open('vocab_data.pkl', 'wb') as fp:
    pickle.dump(vocab, fp)
    print('dictionary saved successfully to file')

with open('label_vocab_data.pkl', 'wb') as fp:
    pickle.dump(label_vocab, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file
